In [41]:
import pandas as pd
import re

years = "2022-2026"
# year = years[0:4]
year = "2026"
file_path = f'DataGovCSVs/TIF_District_Programming_-_{years}.csv'

# Load the workbook and select the active worksheet
df = pd.read_csv(file_path)
df.rename(columns={"TIF Name": "TIF1", "Time Period":year, "Tme Period":year}, inplace=True)
df

,TIF Number,TIF1,2026,Type,Description,Amount,Designation Date,Expiration Date
0,T-111,105th/Vincennes,2022,Current Obligations,Program Administration,-14000,10/03/2001,12/31/2025
1,T-111,105th/Vincennes,2022,Revenue,Property tax,1705000,10/03/2001,12/31/2025
2,T-111,105th/Vincennes,2022,Revenue,Property Tax - 2022 - Collection Variance,-527000,10/03/2001,12/31/2025
3,T-111,105th/Vincennes,2023,Proposed Projects,AIS - Fire Hook And Ladder 24 - 10400 S Vincen...,-200000,10/03/2001,12/31/2025
4,T-111,105th/Vincennes,2023,Proposed Projects,AIS - Library - West Pullman - 830 W 119th St,-200000,10/03/2001,12/31/2025
...,...,...,...,...,...,...,...,...
5272,T-065,Woodlawn,Fund / Project Balances,Current Obligations,CDOT - Street resurfacing - 62nd Place / 62nd ...,-133200,01/20/1999,12/31/2035
5273,T-065,Woodlawn,Fund / Project Balances,Current Obligations,DPD - Professional services,-1900,01/20/1999,12/31/2035
5274,T-065,Woodlawn,Fund / Project Balances,Fund Balance,FY 2021 Year End Available Fund Balance,19260500,01/20/1999,12/31/2035
5275,T-065,Woodlawn,Fund / Project Balances,Current Obligations,NIF,-707400,01/20/1999,12/31/2035


In [42]:
df = df[  (df['Type'] == "Transfers Between TIF Districts")
        & (df[year] == year)]
df

,TIF Number,TIF1,2026,Type,Description,Amount,Designation Date,Expiration Date
37,T-176,107th/Halsted,2026,Transfers Between TIF Districts,From 119th/I-57 (Invest South West Program),1000000,03/05/2014,12/31/2038
210,T-125,119th/I-57,2026,Transfers Between TIF Districts,To 107th/Halsted (Invest South West Program),-1000000,11/06/2002,12/31/2026
1607,T-063,Canal/Congress,2026,Transfers Between TIF Districts,To Roosevelt Clark (Infra-Structure - Roosevel...,-24285800,11/12/1998,12/31/2034
3351,T-075,Madison/Austin Corridor,2026,Transfers Between TIF Districts,From Northwest Ind. Corr. (Invest South West P...,1500000,09/29/1999,12/31/2035
3520,T-095,Midwest,2026,Transfers Between TIF Districts,To Ogden/Pulaski (Invest South West Program),-6408000,05/17/2000,12/31/2036
3765,T-166,North Pullman,2026,Transfers Between TIF Districts,From Roseland/Michigan (RDA - Whole Foods Dist...,145000,06/30/2009,12/31/2033
3858,T-064,Northwest Industrial Corridor,2026,Transfers Between TIF Districts,To Madison/Austin (Invest South West Program),-1500000,12/02/1998,12/31/2034
3932,T-161,Ogden/Pulaski,2026,Transfers Between TIF Districts,From Midwest (Invest South West Program),6408000,04/09/2008,12/31/2032
4422,T-184,Roosevelt Clark,2026,Transfers Between TIF Districts,From Canal/Congress (Infra-Structure - Rooseve...,24285800,04/10/2019,12/31/2043
4605,T-113,Roseland/Michigan,2026,Transfers Between TIF Districts,To North Pullman (RDA - Whole Foods Distribution),-145000,01/16/2002,12/31/2026


In [43]:
def extract_TIF2(row) -> str:
    # print(row)
    description = row["Description"]
    if "To" in description or "From" in description:
        return str(re.findall(r"(?:To|From)\s(.*?)\s\(", description)[0])
    elif "CDOT" in description:
        amount = row["Amount"]
        cdots = (df[df["Description"].str.contains("CDOT")])
        other_row = cdots[(description == cdots["Description"]) & (amount*-1 == cdots["Amount"])]
        return str(other_row["TIF1"].iloc[0])
    elif "SBIF" in description:
        amount = row["Amount"]
        sbifs = (df[df["Description"].str.contains("SBIF")])
        other_row = sbifs[(description == sbifs["Description"]) & (amount*-1 == sbifs["Amount"])]
        return str(other_row["TIF1"].iloc[0])
    elif "Port To" in description or "Port to" in description:
        amount = row["Amount"]
        other_row = df[(description == df["Description"]) & (amount*-1 == df["Amount"])]
        return str(other_row["TIF1"].iloc[0])
    elif "Port from" in description:
        amount = row["Amount"]
        other_row = df[(description == df["Description"]) & (amount*-1 == df["Amount"])]
        return str(other_row["TIF1"].iloc[0])
    else:
        raise ValueError

def extract_Project(row) -> str:
    # print(row)
    description = row["Description"]
    if "To" in description or "From" in description:
        return str(re.findall(r"\((.*?)\)", description)[0])
    elif "CDOT" in description or "Port To" in description or "Port to" in description or "Port from" in description:
        return str(re.findall(r"-\s*(.*)", description)[0])
    elif "SBIF" in description:
        return "SBIF"
    else:
        raise ValueError

def extract_Direction(row) -> str:
    description = row["Description"]
    if "To" in description or "From" in description:
        return str(re.findall(r"^\b(\w+)\b", description)[0])
    elif "CDOT" in description or "Port To" in description or "Port to" in description or "Port from" in description:
        return "To" if row["Amount"] < 0 else "From"
    elif "SBIF" in description:
        amount = row["Amount"]
        return "To" if amount < 0 else "From"
    else:
        raise ValueError

In [44]:
# df['TIF2'] = df['Description'].str.extract(r"(?:To|From)\s(.*?)\s\(")
df['TIF2'] = df.apply(extract_TIF2, axis=1)
# df['Project'] = df['Description'].str.extract(r"\((.*?)\)")
df['Project'] = df.apply(extract_Project, axis=1)
# df['Direction'] = df['Description'].str.extract(r"^\b(\w+)\b")
df['Direction'] = df.apply(extract_Direction, axis=1)

C:\Users\email\AppData\Local\Temp\ipykernel_27772\4201823378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TIF2'] = df.apply(extract_TIF2, axis=1)
C:\Users\email\AppData\Local\Temp\ipykernel_27772\4201823378.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Project'] = df.apply(extract_Project, axis=1)
C:\Users\email\AppData\Local\Temp\ipykernel_27772\4201823378.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [45]:
for val in df['Direction']:
    if str(val) != "To" and str(val) != "From":
        print(val)    

In [46]:
df.drop(columns=["Description", "Designation Date", "Expiration Date"], inplace=True)
df = df[["TIF Number","TIF1","TIF2","Direction","Amount","Project"]]

C:\Users\email\AppData\Local\Temp\ipykernel_27772\1704854201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["Description", "Designation Date", "Expiration Date"], inplace=True)


In [47]:
df

,TIF Number,TIF1,TIF2,Direction,Amount,Project
37,T-176,107th/Halsted,119th/I-57,From,1000000,Invest South West Program
210,T-125,119th/I-57,107th/Halsted,To,-1000000,Invest South West Program
1607,T-063,Canal/Congress,Roosevelt Clark,To,-24285800,Infra-Structure - Roosevelt Clark TIF
3351,T-075,Madison/Austin Corridor,Northwest Ind. Corr.,From,1500000,Invest South West Program
3520,T-095,Midwest,Ogden/Pulaski,To,-6408000,Invest South West Program
3765,T-166,North Pullman,Roseland/Michigan,From,145000,RDA - Whole Foods Distribution
3858,T-064,Northwest Industrial Corridor,Madison/Austin,To,-1500000,Invest South West Program
3932,T-161,Ogden/Pulaski,Midwest,From,6408000,Invest South West Program
4422,T-184,Roosevelt Clark,Canal/Congress,From,24285800,Infra-Structure - Roosevelt Clark TIF
4605,T-113,Roseland/Michigan,North Pullman,To,-145000,RDA - Whole Foods Distribution


In [48]:
df.to_csv(f"DataGovCSVs/TIF_Transfers_{year}.csv", index=False)